In [232]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.misc
import glob
import sys
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes

tf.logging.set_verbosity(tf.logging.INFO)

In [233]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [234]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
#             print (img_arr.shape)
        img_arr = get_img_array(f)
#         img_arr = get_img_array(f).reshape([-1, 32, 32, 3])
        img_arr = np.float32(img_arr / 255.0)
        images.append(img_arr)
    images = np.stack(images)
    print (images.shape)
    return images

def get_all_paths(folder):
    """
    return a list of directory of all imgs in folder
    """
    files = get_files(folder)
    paths = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        paths.append(folder + f)
    
    return paths
    
def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
#     paths = get_all_paths(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)
    
def get_batch(X, y, batch_size):
    """
    Return minibatch of samples and labels
        
    :param X, y: samples and corresponding labels
    :parma batch_size: minibatch size
    :returns: (tuple) X_batch, y_batch
    """
    m = X.shape[0]
    index = np.random.randint(0, m - batch_size)
    X_batch = X[index:(index + batch_size)]
    y_batch = y[index:(index + batch_size)]

    return X_batch, y_batch

def Z_ScoreNorm(x):
    mu = np.average(x)
    std = np.std(x)
    x=(x-mu)/std
    return x

In [235]:
# Right now we have the following strcuture: Conv1 -> Norm1 -> Pool1 -> Conv2 -> Norm2 -> Pool2 -> Dense 
# -> Dense -> Logits
def cnn_model_fn(features):
    """Model function for CNN."""
    # Input Layer; has size batchsize*32*32*3
    with tf.variable_scope("data"):
        input_layer = tf.reshape(features, [-1, 32, 32, 3])
    
    # Initialize a regularizer, using L2 regularization
    regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
    # Convolutional Layer #1
    # output size batchsize*32*32*32
    with tf.variable_scope("conv1"):
        conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters = 32,
            kernel_size = [5, 5],
            padding='same',
            activation = tf.nn.relu,
            kernel_regularizer=regularizer)
    # Normalization layer #1:  output size batchsize*32*32*32
    with tf.variable_scope("norm1"):
#         norm1 = tf.layers.batch_normalization(inputs=conv1)
        norm1 = tf.nn.lrn(conv1)
    
    # Pooling Layer #1; output size batchsize*16*16*32
    with tf.variable_scope("pool1"):
        pool1 = tf.layers.max_pooling2d(inputs=norm1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    # conv2 output size: batchsize*16*16*64
    with tf.variable_scope("conv2"):
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu,
            kernel_regularizer=regularizer)
    
     # norm2 output size: batchsize*16*16*64
    with tf.variable_scope("norm2"):
#         norm2 = tf.layers.batch_normalization(inputs=conv2)
        norm2 = tf.nn.lrn(conv2)
    
    # pool2 output size: batchsize*8*8*64
    with tf.variable_scope("pool2"):
        pool2 = tf.layers.max_pooling2d(inputs=norm2, pool_size=[2,2], strides = 2)
    
    # Dense Layer
    # pool2_flat size: batchsize*1024 
    with tf.variable_scope("fully_connect1"):
        pool2_flat = tf.reshape(pool2, [-1, 8*8*64])
    # dense1 size: batchsize*512
    with tf.variable_scope("fully_connect2"):
        dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, kernel_regularizer=regularizer)
    
#     # dense2 size: batchsize*128
#     with tf.variable_scope("fully_connect3"):
#         dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
    # dropout， we will not use dropout here
    
    # Logits Layer
    with tf.variable_scope("logits"):
        y_conv = tf.layers.dense(inputs=dense1, units=10)
    
    global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
    
    return y_conv, global_step
 

In [197]:
# Load the data
data_root_path = '/Users/mrdoggie/Desktop/Columbia/deepLearning/HW1/cifar10-hw1/'
train_data, train_labels = get_train_data(data_root_path) # this may take a few minutes
# test_data_paths = get_images(data_root_path + 'test')

# normalize the data before training.
train_data = Z_ScoreNorm(train_data)
# arrange 10% of the training data to validation set
valid_data = []
valid_labels = []
indexs = np.random.choice(50000, 5000, replace=False)
valid_data = train_data[indexs]
valid_labels = train_labels[indexs]
train_data = np.delete(train_data,indexs, axis = 0)
train_labels = np.delete(train_labels,indexs)

print('Data loading done')


{'horse': 7, 'automobile': 1, 'deer': 4, 'dog': 5, 'frog': 6, 'cat': 3, 'truck': 9, 'ship': 8, 'airplane': 0, 'bird': 2}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
(50000, 32, 32, 3)
Loaded 10000/10000
(10000, 32, 32, 3)
Data loading done


In [237]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 32,32,3])
y = tf.placeholder(tf.float32, [None, 10])

y_conv, global_step = cnn_model_fn(x)

with tf.variable_scope("loss"):
    loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=y_conv)
tf.summary.scalar("Total_loss", loss)

with tf.variable_scope('GD_optimizer'):
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.02)
    gradients = opt.compute_gradients(loss)
    train_step = opt.apply_gradients(gradients, global_step=global_step)
for gv in gradients:
    namespace = str(gv[1])
    if "conv1/conv2d/kernel:0" in namespace:
        tf.summary.histogram("Gradients/conv1", gv[0])
    if "conv2/conv2d/kernel:0" in namespace:
        tf.summary.histogram("Gradients/conv2", gv[0])
    if "fully_connect2/dense/kernel:0" in namespace:
        tf.summary.histogram("Gradients/last_fully_connected", gv[0])
        
#     train_step = opt.minimize(loss, global_step=global_step)
# print (tf.trainable_variables())
# tf.summary.histogram("Gradients", tf.gradients(loss, 'conv1/conv2d/kernel:0'))

with tf.variable_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)
tf.summary.scalar("Accuracy/train", accuracy)

merged=tf.summary.merge_all()
saver=tf.train.Saver()

with tf.Session() as sess:
    SAVE_PATH = "/Users/mrdoggie/Desktop/Columbia/deepLearning/HW2/part1-final-version/"
    print('Saving graph to: %s' % SAVE_PATH)
    train_writer = tf.summary.FileWriter(SAVE_PATH)
    train_writer.add_graph(tf.get_default_graph())
    
    try:
        print("Trying to restore last checkpoint ...")
        last_chk_path=tf.train.latest_checkpoint(checkpoint_dir=SAVE_PATH)
        saver.restore(sess, save_path=last_chk_path)
        print("Restored checkpoint from:", last_chk_path)
    except:
        print("Failed to restore checkpoint. Restarting...")
        sess.run(tf.global_variables_initializer())
    
    one_hot_valid_labels = tf.one_hot(indices=tf.cast(valid_labels, tf.int32), depth=10)
    one_hot_valid_labels = sess.run(one_hot_valid_labels)
    for i in range(3001):
        x_batch, y_batch = get_batch(train_data, train_labels, 128)
        y_batch = tf.one_hot(indices=tf.cast(y_batch, tf.int32), depth=10)
        y_batch = sess.run(y_batch)

        
        if i % 500 == 0:
            data_merged, num_step, train_accuracy, train_loss = sess.run([merged, global_step, accuracy, loss], feed_dict={
                x: x_batch,
                y: y_batch
            })
            
            valid_accuracy = accuracy.eval(feed_dict={
                x: valid_data, y: one_hot_valid_labels})
#             train_accuracy, train_loss = sess.run([accuracy, loss], feed_dict={
#                 x: x_batch,
#                 y: y_batch
#             })
#             train_accuracy = accuracy.eval(feed_dict={
#                 x: x_batch,
#                 y: y_batch})
#             train_loss = loss.eval(feed_dict={
#                 x: x_batch,
#                 y: y_batch})
            print('step: %d, training accuracy: %g, loss: %f, validation accuracy: %f' % (num_step, train_accuracy, train_loss, valid_accuracy))
            summary = tf.Summary(value=[
                tf.Summary.Value(tag="Accuracy/validation", simple_value=valid_accuracy),
            ])
            train_writer.add_summary(data_merged, num_step)
            train_writer.add_summary(summary, num_step)
            # save the process
            saver.save(sess, save_path=SAVE_PATH, global_step=global_step)
            print("Saved checkpoint.")
        train_step.run(feed_dict={x: x_batch, y: y_batch})
        

    print('Final Accuracy on Validation Set: %g' % accuracy.eval(feed_dict={
        x: valid_data, y: one_hot_valid_labels}))

Saving graph to: /Users/mrdoggie/Desktop/Columbia/deepLearning/HW2/part1-final-version/
Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from /Users/mrdoggie/Desktop/Columbia/deepLearning/HW2/part1-final-version/-5000
Restored checkpoint from: /Users/mrdoggie/Desktop/Columbia/deepLearning/HW2/part1-final-version/-5000
step: 5000, training accuracy: 0.875, loss: 0.321993, validation accuracy: 0.723400
Saved checkpoint.
step: 5500, training accuracy: 0.90625, loss: 0.313460, validation accuracy: 0.747000
Saved checkpoint.
step: 6000, training accuracy: 0.921875, loss: 0.288588, validation accuracy: 0.751600
Saved checkpoint.
step: 6500, training accuracy: 0.9375, loss: 0.233372, validation accuracy: 0.754400
Saved checkpoint.
step: 7000, training accuracy: 0.898438, loss: 0.332599, validation accuracy: 0.753200
Saved checkpoint.
step: 7500, training accuracy: 0.96875, loss: 0.182665, validation accuracy: 0.755200
Saved checkpoint.
step: 8000, training accuracy: 

In [ ]:
# 5000 steps with step size = 0.05
# followed with 3000 step size = 0.02

In [240]:
# load test data
x_test = get_images(data_root_path + 'test')
with tf.Session() as sess:
    try:
        print("Trying to restore last checkpoint ...")
        last_chk_path=tf.train.latest_checkpoint(checkpoint_dir=SAVE_PATH)
        saver.restore(sess, save_path=last_chk_path)
        print("Restored checkpoint from:", last_chk_path)
    except:
        print("Failed to restore checkpoint. Restarting...")
        
    predict_raw = y_conv.eval(feed_dict={x: x_test})
    predict_raw = predict_raw.T
    print (predict_raw.shape)
    save_predictions('ans1-jw3564.npy', predict_raw)

Loaded 10000/10000
(10000, 32, 32, 3)
Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from /Users/mrdoggie/Desktop/Columbia/deepLearning/HW2/part1-final-version/-8000
Restored checkpoint from: /Users/mrdoggie/Desktop/Columbia/deepLearning/HW2/part1-final-version/-8000
(10, 10000)


In [241]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-jw3564.npy')
print(loaded_y.shape)

(10, 10000)
